In [20]:
import nltk
nltk.download('punkt')
from dotenv import load_dotenv
load_dotenv()

import os
from lxml import etree

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
os.environ["Gemini_API_key"] = os.getenv('Gemini_API_key')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

from langchain.agents import AgentExecutor
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.google_palm import GooglePalmEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
total_info=""

main_directory_path = 'MedQuAD-master'
for root, dirs, files in os.walk(main_directory_path):
  for file in files:
      if file.endswith('.xml'):  
          file_path = os.path.join(root, file)
          
          try:
              tree = etree.parse(file_path)
              root_element = tree.getroot()
              topic = root_element.find('.//Focus')
              if topic is not None:
                  total_info=total_info +" \n " +topic.text+ " \n "
              
              for specific_element in root_element.iter("QAPair"):
                  quest = specific_element.find('.//Question')
                  answ= specific_element.find('.//Answer')
                  question_text = quest.text
                  answer_text = answ.text
                  qtype = quest.attrib.get('qtype', 'No qtype found')
                  if topic is not None:
                    total_info=total_info +" (Question Type: "+qtype+") Question: "+question_text+ " Answer: "+answer_text 
            
              total_info = total_info+" \n "
          except etree.XMLSyntaxError as e:
              print(f"Error parsing {file_path}: {e}")

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
documents=text_splitter.create_documents([total_info[0:6500]])
palm_embeddings=GooglePalmEmbeddings()

c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
vector = FAISS.from_documents(documents, palm_embeddings)

In [17]:
retriever = vector.as_retriever()

In [18]:
retriever_tool = create_retriever_tool(
    retriever,
    "medical_tool",
    "searches the medical documents"
)

In [19]:
tools = [retriever_tool]

In [22]:
model = ChatGroq(model="llama3-8b-8192")

In [23]:
model_with_tools = model.bind_tools(tools)

In [24]:
prompt=PromptTemplate.from_template(
"""
You are a mecial chatbot and answer the user query in short based on the tool. If some other questions 
are asked other than medical then say something like you are only designed to answer medical questions only.
question: {input}
answer:''
{agent_scratchpad}
"""
)

In [25]:
agent = create_tool_calling_agent(model, tools, prompt)

In [27]:
agent_executor = AgentExecutor(agent=agent, tools=tools, output_parser=StrOutputParser())

In [29]:
ans=agent_executor.invoke({"input":"Adult Acute Lymphoblastic"})  

In [31]:
ans

{'input': 'Adult Acute Lymphoblastic',
 'output': 'Adult Acute Lymphoblastic Leukemia (ALL) is a type of cancer in which the bone marrow makes too many lymphocytes (a type of white blood cell).'}